# Benchmark

**!** This notebook has to be run after the following notebooks:
- [dataset.ipynb](dataset.ipynb)
- [scoring.ipynb](scoring.ipynb)
- [model.ipynb](model.ipynb)

In [1]:
import model
import dataset
import scoring

In [2]:
models = model.load_models()

| using constant padding
| using scales: [0.8333333333333334, 1.0, 1.2]
| using ordinary correlation
load pretrained model from models/SiamSE/checkpoint_vot.pth
remove prefix "module."
missing keys:set()
unused checkpoint keys:set()


/mnt/c/Users/33652/Documents/ING 2 Prog/DeepNN/venv_dnn/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/c/Users/33652/Documents/ING 2 Prog/DeepNN/venv_dnn/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
datasets = dataset.load_datasets()

Error while loading dataset David gt and image files have different length
Error while loading dataset Football1 gt and image files have different length
Error while loading dataset Jogging1 gt and image files have different length
Error while loading dataset Jogging2 gt and image files have different length
Error while loading dataset Subway gt and image files have different length


In [4]:
models.keys(), datasets.keys()

(dict_keys(['SEsiamFC', 'AAA']), dict_keys(['mytc128', 'myvot2021']))

# Tracking

In [5]:
import cv2
import numpy as np

In [27]:
def track_video(tracker, video, verbose=False):
    start_frame, toc = 0, 0

    pred = []
    image_files, gt = video['image_files'], video['gt']

    for f, image_file in enumerate(image_files):
        tic = cv2.getTickCount()

        if f == start_frame:  # init
            tracker.initialize(image_file, np.array(gt[f]))
            pred.append(gt[f])

        elif f > start_frame:  # tracking
            pred_bbox = tracker.track(image_file)
            print("{} gt: {} pred: {}".format(f, gt[f], pred_bbox))
            b_overlap = scoring.get_precision(gt[f], pred_bbox)
            if b_overlap > 0:
                pred.append(pred_bbox)
            else:
                pred.append(2)
                start_frame = f + 5
        else:
            pred.append(0)

        toc += cv2.getTickCount() - tic
  
    toc /= cv2.getTickFrequency()

    precisions = [scoring.get_precision(gt[i], pred[i]) for i in range(len(gt))]
    precisions = np.array(precisions)
    mprec = np.mean(precisions)

    success = [scoring.is_success(gt[i], pred[i]) for i in range(len(gt))]
    success = np.array(success)
    msucc = np.mean(success)

    fps = f / toc

    if verbose:
        print('Video: {:12s} Time: {:2.1f}s Speed: {:3.1f}fps mSuccess: {:3.1f} mPrecision {:3.1f}'.format(video['name'], toc, f / toc, msucc, mprec))
    return mprec, msucc, fps

In [28]:
t1 = models['SEsiamFC']
t2 = models['AAA']
d1 = datasets['mytc128']

In [29]:
v1 = d1['Cup']

In [25]:
track_video(t1, v1, verbose=True)

Video: Cup          Time: 34.9s Speed: 8.7fps mSuccess: 1.0 mPrecision 0.8


(0.8127891293539966, 1.0, 8.653954689228593)

In [31]:
track_video(t2, v1, verbose=True)


1 gt: [124, 92, 46, 59] pred: [0. 0. 0. 0.]
7 gt: [123, 93, 46, 57] pred: [0. 0. 0. 0.]
13 gt: [120, 97, 46, 58] pred: [0. 0. 0. 0.]
19 gt: [116, 98, 46, 57] pred: [0. 0. 0. 0.]
25 gt: [122, 94, 46, 56] pred: [0. 0. 0. 0.]
31 gt: [128, 88, 44, 56] pred: [0. 0. 0. 0.]
37 gt: [120, 82, 44, 54] pred: [0. 0. 0. 0.]
43 gt: [118, 92, 42, 54] pred: [0. 0. 0. 0.]
49 gt: [121, 97, 41, 53] pred: [0. 0. 0. 0.]
55 gt: [124, 95, 43, 53] pred: [0. 0. 0. 0.]
61 gt: [122, 97, 43, 52] pred: [0. 0. 0. 0.]
67 gt: [124, 108, 40, 49] pred: [0. 0. 0. 0.]
73 gt: [126, 99, 37, 46] pred: [0. 0. 0. 0.]
79 gt: [125, 84, 38, 46] pred: [0. 0. 0. 0.]
85 gt: [122, 90, 38, 45] pred: [0. 0. 0. 0.]
91 gt: [113, 98, 38, 46] pred: [0. 0. 0. 0.]
97 gt: [104, 105, 37, 47] pred: [0. 0. 0. 0.]
103 gt: [107, 107, 40, 46] pred: [0. 0. 0. 0.]
109 gt: [124, 105, 42, 49] pred: [0. 0. 0. 0.]
115 gt: [136, 103, 41, 49] pred: [0. 0. 0. 0.]
121 gt: [129, 106, 42, 49] pred: [0. 0. 0. 0.]
127 gt: [117, 95, 42, 50] pred: [0. 0. 0. 0.]
1

(0.16831683168316833, 0.16831683168316833, 1.4476557371959673)